In [ ]:
import os

# Define o backend do Keras como TensorFlow
os.environ["KERAS_BACKEND"] = "tensorflow"

# Importações de bibliotecas padrão e de terceiros
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import tensorflow_datasets.public_api as tfds
from tensorflow import keras
from keras import layers, Model, Input
from keras.layers import TextVectorization
from keras import ops
from keras import backend as K  # Importa o backend do Keras


In [ ]:
# ------------------------
# Carregamento e Pré-processamento dos Dados
# ------------------------

# Inicializa uma lista para armazenar os pares de textos (francês e português)
text_pairs = []

# Abre o arquivo 'data.tsv' para leitura, utilizando codificação UTF-8
with open("data.tsv", "r", encoding="utf-8") as f:
    for line in f:
        # Remove espaços extras e divide a linha usando tabulação como separador
        fields = line.strip().split("\t")
        
        # Garante que a linha tenha pelo menos 4 colunas antes de processar
        if len(fields) < 4:
            continue
        
        # Extrai o texto em francês (segunda coluna)
        french = fields[1]
        
        # Extrai o texto em português (quarta coluna) e adiciona tokens de início e fim
        portuguese = "[start] " + fields[3] + " [end]"
        
        # Adiciona o par (francês, português) à lista
        text_pairs.append((french, portuguese))

# Exibe o primeiro par para verificação
print(text_pairs[0])

# Embaralha os pares de textos para garantir aleatoriedade na divisão dos conjuntos
random.shuffle(text_pairs)

# Define o número de amostras para validação (15% do total)
num_val_samples = int(0.15 * len(text_pairs))

# Define o número de amostras para treinamento (o restante após a divisão para validação e teste)
num_train_samples = len(text_pairs) - 2 * num_val_samples

# Divide os pares em conjuntos de treinamento, validação e teste
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

# Exibe o número total de pares e a distribuição entre treinamento, validação e teste
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

In [ ]:
# Define os caracteres que serão removidos durante a padronização dos textos
strip_chars = string.punctuation + "«" + "»"
strip_chars = strip_chars.replace("[", "")  # Remove colchete de abertura da lista de caracteres
strip_chars = strip_chars.replace("]", "")  # Remove colchete de fechamento da lista de caracteres

# Parâmetros para a vetorização dos textos e treinamento do modelo
vocab_size = 25000          # Tamanho máximo do vocabulário para vetorização
sequence_length = 20        # Comprimento máximo das sequências de entrada
batch_size = 64             # Tamanho do lote (batch) durante o treinamento

def custom_standardization(input_string):
    """
    Padroniza o texto convertendo para minúsculas e removendo caracteres indesejados.

    Args:
        input_string (Tensor): Texto de entrada a ser padronizado.

    Returns:
        Tensor: Texto padronizado com caracteres removidos.
    """
    lowercase = tf_strings.lower(input_string)  # Converte o texto para minúsculas
    return tf_strings.regex_replace(
        lowercase, 
        "[%s]" % re.escape(strip_chars),  # Remove os caracteres definidos em 'strip_chars'
        ""
    )

# Vetorização dos textos em francês
french_vectorization = TextVectorization(
    max_tokens=vocab_size,                # Número máximo de tokens no vocabulário
    output_mode="int",                    # Saída como inteiros (índices dos tokens)
    output_sequence_length=sequence_length  # Comprimento fixo da sequência
)

# Vetorização dos textos em português com padronização personalizada
portuguese_vectorization = TextVectorization(
    max_tokens=vocab_size,                  # Número máximo de tokens no vocabulário
    output_mode="int",                      # Saída como inteiros (índices dos tokens)
    output_sequence_length=sequence_length + 1,  # Comprimento fixo (com 1 extra para o token final)
    standardize=custom_standardization      # Função de padronização personalizada
)

# Extrai os textos em francês e português dos pares de treinamento
train_french_texts = [pair[0] for pair in train_pairs]
train_portuguese_texts = [pair[1] for pair in train_pairs]

# Ajusta (adapta) as camadas de vetorização aos textos de treinamento
french_vectorization.adapt(train_french_texts)
portuguese_vectorization.adapt(train_portuguese_texts)


In [ ]:
def format_dataset(french, portuguese):
    """
    Formata os pares de textos em francês e português para o modelo seq2seq.

    Args:
        french (Tensor): Textos em francês.
        portuguese (Tensor): Textos em português.

    Returns:
        Tuple: Um dicionário com as entradas do encoder e do decoder, 
               e o alvo do decoder para o treinamento.
    """
    # Vetoriza o texto em francês (entrada do encoder)
    french = french_vectorization(french)
    
    # Vetoriza o texto em português (entrada e alvo do decoder)
    portuguese = portuguese_vectorization(portuguese)
    
    return (
        {
            "encoder_inputs": french,  # Entrada do encoder
            "decoder_inputs": portuguese[:, :-1],  # Entrada do decoder (sem o último token)
        },
        portuguese[:, 1:]  # Alvo do decoder (sem o primeiro token)
    )


def make_dataset(pairs):
    """
    Cria um dataset formatado a partir dos pares de texto.

    Args:
        pairs (list): Lista de tuplas contendo os textos em francês e português.

    Returns:
        tf.data.Dataset: Dataset formatado para o treinamento e validação.
    """
    # Separa os textos em francês e português dos pares
    french_texts, portuguese_texts = zip(*pairs)
    
    # Cria um dataset TensorFlow a partir dos textos
    dataset = tf_data.Dataset.from_tensor_slices(
        (list(french_texts), list(portuguese_texts))
    )
    
    # Agrupa os dados em lotes (batch)
    dataset = dataset.batch(batch_size)
    
    # Aplica a função de formatação para preparar as entradas e alvos
    dataset = dataset.map(format_dataset)
    
    # Armazena em cache, embaralha e pré-carrega os dados para otimização
    return dataset.cache().shuffle(2048).prefetch(16)


# Cria os datasets de treinamento e validação utilizando os pares correspondentes
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)


In [ ]:
# ------------------------
# Componentes do Modelo com Regularização Aprimorada
# ------------------------

class PositionalEmbedding(layers.Layer):
    """
    Camada personalizada para aplicar embeddings de tokens combinados com embeddings posicionais.
    Isso permite que o modelo entenda a ordem dos tokens na sequência.
    """
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        """
        Inicializa a camada de embeddings posicionais.

        Args:
            sequence_length (int): Comprimento máximo da sequência.
            vocab_size (int): Tamanho do vocabulário (número máximo de tokens).
            embed_dim (int): Dimensão do vetor de embedding.
            **kwargs: Argumentos adicionais para a classe base `Layer`.
        """
        super().__init__(**kwargs)
        
        # Embedding para os tokens (representação vetorial de cada palavra/token)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, 
            output_dim=embed_dim
        )
        
        # Embedding para a posição de cada token na sequência
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, 
            output_dim=embed_dim
        )
        
        # Armazena os parâmetros para reutilização e configuração
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        """
        Combina embeddings dos tokens com seus respectivos embeddings posicionais.

        Args:
            inputs (Tensor): Sequência de tokens (shape: batch_size, sequence_length).

        Returns:
            Tensor: Embedding combinado de tokens e posições (shape: batch_size, sequence_length, embed_dim).
        """
        # Obtém o comprimento da sequência (última dimensão da entrada)
        length = ops.shape(inputs)[-1]
        
        # Cria um tensor representando as posições (0, 1, 2, ..., length - 1)
        positions = ops.arange(0, length, 1)
        
        # Aplica o embedding nos tokens
        embedded_tokens = self.token_embeddings(inputs)
        
        # Aplica o embedding nas posições
        embedded_positions = self.position_embeddings(positions)
        
        # Combina o embedding dos tokens com o embedding das posições
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        """
        Gera uma máscara para ignorar o padding (valores iguais a 0).

        Args:
            inputs (Tensor): Sequência de entrada.
            mask (Tensor, opcional): Máscara existente.

        Returns:
            Tensor: Máscara onde valores diferentes de zero são marcados como válidos.
        """
        # Cria a máscara usando `ops.not_equal` para detectar tokens válidos (diferentes de 0)
        return ops.not_equal(inputs, 0)

    def get_config(self):
        """
        Retorna a configuração da camada para permitir a serialização do modelo.

        Returns:
            dict: Configurações da camada, incluindo comprimento da sequência, 
                  tamanho do vocabulário e dimensão do embedding.
        """
        config = super().get_config()
        config.update({
            "sequence_length": self.sequence_length,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim,
        })
        return config


In [ ]:
# Transformer Encoder com Dropout adicionado nas camadas de atenção e feedforward
class TransformerEncoder(layers.Layer):
    """
    Encoder do Transformer que aplica atenção multi-cabeça, normalização em camadas
    e redes feedforward, com regularização por Dropout.
    """
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.1, **kwargs):
        """
        Inicializa o Transformer Encoder.

        Args:
            embed_dim (int): Dimensão dos embeddings de entrada.
            dense_dim (int): Dimensão da rede feedforward interna.
            num_heads (int): Número de cabeças de atenção.
            dropout_rate (float): Taxa de dropout para regularização.
            **kwargs: Argumentos adicionais para a classe base `Layer`.
        """
        super().__init__(**kwargs)

        # Parâmetros da camada
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        # Camada de Atenção Multi-Cabeça
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )

        # Dropout aplicado após a camada de atenção
        self.dropout_att = layers.Dropout(dropout_rate)

        # Rede feedforward composta por duas camadas densas
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])

        # Dropout aplicado após a rede feedforward
        self.dropout_ffn = layers.Dropout(dropout_rate)

        # Normalização em camadas aplicada após a atenção e o feedforward
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

        # Indica que a camada suporta mascaramento (útil para lidar com padding)
        self.supports_masking = True

    def call(self, inputs, mask=None, training=False):
        """
        Executa a passagem dos dados pela camada Transformer Encoder.

        Args:
            inputs (Tensor): Tensor de entrada (batch_size, sequence_length, embed_dim).
            mask (Tensor, opcional): Máscara para ignorar posições de padding.
            training (bool, opcional): Indica se a camada está em modo de treinamento.

        Returns:
            Tensor: Saída do encoder após atenção, feedforward e normalização.
        """
        # Se uma máscara for fornecida, ajusta o formato para ser compatível com a atenção
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        # Aplica a atenção multi-cabeça com a máscara de padding (se existir)
        attention_output = self.attention(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=padding_mask
        )

        # Aplica Dropout após a atenção (apenas durante o treinamento)
        attention_output = self.dropout_att(attention_output, training=training)

        # Normaliza a soma residual entre a entrada e a saída da atenção
        proj_input = self.layernorm_1(inputs + attention_output)

        # Passa pela rede feedforward
        proj_output = self.dense_proj(proj_input)

        # Aplica Dropout após o feedforward (apenas durante o treinamento)
        proj_output = self.dropout_ffn(proj_output, training=training)

        # Normaliza a soma residual entre o input normalizado e a saída do feedforward
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        """
        Retorna a configuração da camada para permitir a serialização do modelo.

        Returns:
            dict: Configurações da camada, incluindo embed_dim, dense_dim e num_heads.
        """
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config


In [ ]:
# Transformer Decoder com Dropout adicionado nas subcamadas de atenção e na rede feedforward
class TransformerDecoder(layers.Layer):
    """
    Decoder do Transformer que aplica atenção causal, cross-attention com a saída do encoder,
    normalização em camadas e redes feedforward, com regularização por Dropout.
    """
    def __init__(self, embed_dim, latent_dim, num_heads, dropout_rate=0.1, **kwargs):
        """
        Inicializa o Transformer Decoder.

        Args:
            embed_dim (int): Dimensão dos embeddings de entrada.
            latent_dim (int): Dimensão da rede feedforward interna.
            num_heads (int): Número de cabeças de atenção.
            dropout_rate (float): Taxa de dropout para regularização.
            **kwargs: Argumentos adicionais para a classe base `Layer`.
        """
        super().__init__(**kwargs)

        # Parâmetros do decoder
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads

        # Primeira camada de Atenção Multi-Cabeça (Self-Attention com máscara causal)
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.dropout_att1 = layers.Dropout(dropout_rate)

        # Segunda camada de Atenção Multi-Cabeça (Cross-Attention com saída do encoder)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.dropout_att2 = layers.Dropout(dropout_rate)

        # Rede feedforward com ativação ReLU
        self.dense_proj = keras.Sequential([
            layers.Dense(latent_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.dropout_ffn = layers.Dropout(dropout_rate)

        # Normalização em camadas após cada subcamada do decoder
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        # Indica que a camada suporta mascaramento (útil para lidar com padding)
        self.supports_masking = True

    def call(self, inputs, mask=None, training=False):
        """
        Executa a passagem dos dados pela camada Transformer Decoder.

        Args:
            inputs (tuple): Tupla contendo (decoder_inputs, encoder_outputs).
            mask (tuple, opcional): Máscaras para o decoder e o encoder.
            training (bool, opcional): Indica se a camada está em modo de treinamento.

        Returns:
            Tensor: Saída do decoder após atenção, feedforward e normalização.
        """
        decoder_inputs, encoder_outputs = inputs

        # Cria a máscara causal para evitar que o decoder veja tokens futuros
        causal_mask = self.get_causal_attention_mask(decoder_inputs)

        # Se uma máscara for fornecida, separa as máscaras do decoder e do encoder
        if mask is None:
            decoder_padding_mask, encoder_padding_mask = None, None
        else:
            decoder_padding_mask, encoder_padding_mask = mask

        # Primeira Atenção Multi-Cabeça (Self-Attention com máscara causal)
        attention_output_1 = self.attention_1(
            query=decoder_inputs,
            value=decoder_inputs,
            key=decoder_inputs,
            attention_mask=causal_mask,
            query_mask=decoder_padding_mask,
        )
        attention_output_1 = self.dropout_att1(attention_output_1, training=training)

        # Soma residual e normalização após a primeira atenção
        out_1 = self.layernorm_1(decoder_inputs + attention_output_1)

        # Segunda Atenção Multi-Cabeça (Cross-Attention com saída do encoder)
        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            query_mask=decoder_padding_mask,
            key_mask=encoder_padding_mask,
        )
        attention_output_2 = self.dropout_att2(attention_output_2, training=training)

        # Soma residual e normalização após a cross-attention
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        # Passagem pela rede feedforward
        proj_output = self.dense_proj(out_2)
        proj_output = self.dropout_ffn(proj_output, training=training)

        # Soma residual e normalização após a rede feedforward
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        """
        Gera uma máscara causal para a atenção, impedindo o acesso a tokens futuros.

        Args:
            inputs (Tensor): Sequência de entrada do decoder.

        Returns:
            Tensor: Máscara causal com shape (batch_size, seq_length, seq_length).
        """
        input_shape = tf.shape(inputs)
        batch_size, seq_length = input_shape[0], input_shape[1]

        # Cria uma matriz onde cada posição i só pode ver até a posição j (i >= j)
        i = tf.range(seq_length)[:, None]
        j = tf.range(seq_length)
        mask = tf.cast(i >= j, dtype="int32")

        # Ajusta o formato da máscara para ser compatível com a atenção multi-cabeça
        mask = tf.reshape(mask, (1, seq_length, seq_length))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.convert_to_tensor([1, 1])], axis=0
        )
        return tf.tile(mask, mult)

    def get_config(self):
        """
        Retorna a configuração da camada para permitir a serialização do modelo.

        Returns:
            dict: Configurações da camada, incluindo embed_dim, latent_dim e num_heads.
        """
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config


In [ ]:
# ------------------------
# Construção do Modelo Transformer
# ------------------------

# Definição dos hiperparâmetros do modelo
embed_dim = 128           # Dimensão do embedding para os tokens
latent_dim = 1024         # Dimensão da rede feed-forward dentro dos blocos do Transformer
num_heads = 6             # Número de cabeças de atenção na Multi-Head Attention
sequence_length = 20      # Comprimento máximo da sequência de entrada
vocab_size_model = 15000  # Tamanho do vocabulário usado no modelo

# ------------------------
# Construção do Encoder
# ------------------------

# Entrada do encoder: sequência de tokens inteiros (shape: batch_size, sequence_length)
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

# Aplica a camada de embedding posicional para adicionar informações sobre a ordem dos tokens
x = PositionalEmbedding(sequence_length, vocab_size_model, embed_dim)(encoder_inputs)

# Passa os embeddings pelo Transformer Encoder
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

# Define o modelo do encoder, mapeando as entradas para as saídas processadas
encoder = keras.Model(encoder_inputs, encoder_outputs)

# ------------------------
# Construção do Decoder (usando Teacher Forcing)
# ------------------------

# Entrada do decoder: sequência de tokens de destino (shape: batch_size, sequence_length)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")

# Aplica a camada de embedding posicional nos inputs do decoder
x = PositionalEmbedding(sequence_length, vocab_size_model, embed_dim)(decoder_inputs)

# Passa os embeddings pelo Transformer Decoder, utilizando as saídas do encoder
x = TransformerDecoder(embed_dim, latent_dim, num_heads)([x, encoder_outputs])

# Camada densa final para gerar as previsões de tokens (com ativação softmax)
decoder_outputs = layers.Dense(vocab_size_model, activation="softmax")(x)

# ------------------------
# Definição do Modelo Final Transformer
# ------------------------

# Cria o modelo Transformer completo, com entradas para o encoder e decoder
transformer = keras.Model(
    {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs},
    decoder_outputs,
    name="transformer",
)

# Exibe o resumo da arquitetura do modelo, mostrando as camadas e parâmetros
transformer.summary()


In [ ]:
# ------------------------
# Compilação do Modelo com Otimizador Adam e Scheduler de Aprendizado
# ------------------------

# Definição de um scheduler para a taxa de aprendizado adaptativa
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,  # Taxa de aprendizado inicial
    decay_steps=10000,           # Número de passos antes de aplicar o decaimento
    decay_rate=0.9,              # Fator de decaimento exponencial
    staircase=True               # O decaimento ocorre em "degraus" (não contínuo)
)

# Otimizador Adam com a taxa de aprendizado controlada pelo scheduler
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compilação do modelo Transformer
transformer.compile(
    optimizer=optimizer,  # Otimizador Adam com taxa de aprendizado adaptativa
    loss=keras.losses.SparseCategoricalCrossentropy(ignore_class=0),  
    # Função de perda com entropia cruzada esparsa (ignora o índice 0, usado para padding)
    metrics=["accuracy"]  # Métrica de acurácia para avaliar o desempenho durante o treinamento
)


In [ ]:
# ------------------------
# Treinamento do Modelo Transformer
# ------------------------

# Define o número de épocas para o treinamento do modelo
epochs = 30  # Considere aumentar este valor ou usar early stopping para evitar overfitting

# Inicia o processo de treinamento do modelo
history = transformer.fit(
    train_ds,              # Dataset de treinamento
    epochs=epochs,         # Número de épocas para o treinamento
    validation_data=val_ds  # Dataset de validação para monitorar o desempenho
)


('Je ne supporte pas ce type.', '[start] Eu não suporto esse tipo. [end]')
33030 total pairs
23122 training pairs
4954 validation pairs
4954 test pairs


Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 128) │  1,922,560 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 128) │  1,922,560 │ decoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 128) │    659,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, None)      │          0 │ decoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, None, 128) │  1,055,360 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
│                     │                   │            │ not_equal_3[0][0… │
│                     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, None,      │  1,935,000 │ transformer_deco… │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,494,936 (28.59 MB)

 Trainable params: 7,494,936 (28.59 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
362/362 ━━━━━━━━━━━━━━━━━━━━ 123s 328ms/step - accuracy: 0.0637 - loss: 6.4672 - val_accuracy: 0.1304 - val_loss: 4.2834
Epoch 2/30
362/362 ━━━━━━━━━━━━━━━━━━━━ 124s 341ms/step - accuracy: 0.1479 - loss: 3.7420 - val_accuracy: 0.1775 - val_loss: 3.3135
Epoch 3/30
362/362 ━━━━━━━━━━━━━━━━━━━━ 123s 340ms/step - accuracy: 0.2043 - loss: 2.4399 - val_accuracy: 0.1997 - val_loss: 2.9113
Epoch 4/30
129/362 ━━━━━━━━━━━━━━━━━━━━ 1:14 319ms/step - accuracy: 0.2455 - loss: 1.5932

KeyboardInterrupt: 